# Add comments to a non-annotated model based on chem_annotated.inp

For an RMG job, under the `chemkin` folder, there will be a couple of `chem_XXX.inp` file and one `chem_annotated.inp`. The latter contains all the species and reactions found and is the only one with complete information about thermo and kinetics estimation details. This notebook help to annotate the intermediate models based on the `chem_annotated.inp`

In [ ]:
import os

from tqdm.notebook import tqdm

from rmgpy.chemkin import load_chemkin_file
from rmgpy.chemkin import save_chemkin_file, save_species_dictionary, save_transport_file

In [ ]:
model_path = "chemkin"
model1 = 'chem_annotated.inp'
model2 = 'chem0259.inp'


model_path = os.path.abspath(model_path)
spcs1, rxns1 = load_chemkin_file(os.path.join(model_path, model1,),
                                 os.path.join(model_path, 'species_dictionary.txt'),)
spcs2, rxns2 = load_chemkin_file(os.path.join(model_path, model2,),
                                 os.path.join(model_path, 'species_dictionary.txt'),)

Update species

In [ ]:
updated_spcs = [spc for spc in spcs1 if spc.index <= 0]  # noble gas elements have weird species index.
spcs_dict1 = {spc.index: spc for spc in spcs1}  # Build species dict for model 1

# If the RMG runs correctly, all models should share the same sequence for species and reactions.
# This for loop checks whether the above statement is true, though have no solution when it is false.
for spc in tqdm(spcs2):
    index = spc.index
    if index <= 0:
        continue
    elif index not in spcs_dict1.keys():
        print('Index inconsistency!')
        print(spc)
        continue
    found_spc = spcs_dict1[index]
    if found_spc.smiles == spc.smiles:
        updated_spcs.append(found_spc)
    elif found_spc.is_isomorphic(spc):
        updated_spcs.append(found_spc)
    else:
        print('Species inconsistency!')
        print(f'Index {index}')
        print(spc, found_spc)

Update reactions

In [ ]:
updated_rxns = []
rxns_dict1 = {rxn.index: rxn for rxn in rxns1}
for rxn in tqdm(rxns2):
    index = rxn.index
    if index not in rxns_dict1.keys():
        print('Index inconsistency!')
        print(rxn)
        continue
    found_rxn = rxns_dict1[index]
    r1 = set([r.smiles for r in found_rxn.reactants])
    r2 = set([r.smiles for r in rxn.reactants])
    p1 = set([p.smiles for p in found_rxn.products])
    p2 = set([p.smiles for p in rxn.products])
    if r1 == r2 and p1 == p2:
        updated_rxns.append(found_rxn)
    elif found_rxn.is_isomorphic(rxn):
        updated_rxns.append(found_rxn)
    else:
        print('Reaction inconsistency!')
        print(f'Index {index}')
        print(rxn, found_rxn)

Sort species and reactions based on their indexes

In [ ]:
updated_spcs.sort(key=lambda spc: spc.index)
updated_rxns.sort(key=lambda rxn: rxn.index)

Save the model

In [ ]:
############### INPUT REQUIRED  ###################
# You can modify the variables below to change the save location
save_dir = model_path
save_chem_name = 'chem_annotated_new.inp'
save_dict_name = 'species_dictionary_new.txt'
#########################################
os.makedirs(save_dir, exist_ok=True)
save_chemkin_file(os.path.join(save_dir, save_chem_name), updated_spcs, updated_rxns, verbose=True)
save_species_dictionary(os.path.join(save_dir, save_dict_name), updated_spcs)
# save_transport_file(os.path.join(save_dir, 'tran.dat'), spcs)

Test readability

In [ ]:
load_chemkin_file(os.path.join(save_dir, 'chem_annotated_new.inp'),
                  os.path.join(save_dir, 'species_dictionary_new.txt'),)